In [2]:
import findspark
findspark.init()

In [3]:
"""
Only use this command to change package information during pyspark shell launch. This cell is not working
"""
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = \
 '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'
#,org.apache.spark:spark-avro_2.12:3.5.0 pyspark-shell' 

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.streaming import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.streaming import StreamingContext
#import os

spark = SparkSession.builder.master("local").appName("kafkastream").getOrCreate()
sc = spark.sparkContext
ssc = StreamingContext(sc,60)  
## Create a local StreamingContext with two working thread and batch interval of 60 second

# Create a DStream that will connect to hostname:port, like localhost:9999
#lines = ssc.socketTextStream("localhost", 9999)

C:\Program Files\spark3.5\python\pyspark\streaming\context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


+----+-----+
|word|count|
+----+-----+
+----+-----+

+-----------+-----+
|       word|count|
+-----------+-----+
|        for|    1|
|    example|    1|
|      hello|    1|
|       with|    1|
|integration|    1|
|  streaming|    1|
|         is|    1|
|      Kafka|    1|
|    testing|    1|
|     count.|    1|
|      Hello|    1|
|      spark|    1|
|        the|    1|
|       word|    1|
| streaming.|    1|
|     sample|    1|
|      world|    1|
|       This|    1|
|      Spark|    1|
|       text|    1|
+-----------+-----+
only showing top 20 rows

+-----------+-----+
|       word|count|
+-----------+-----+
|        for|    2|
|    example|    2|
|      hello|    2|
|       with|    2|
|integration|    2|
|  streaming|    2|
|         is|    2|
|      Kafka|    2|
|    testing|    2|
|     count.|    2|
|      spark|    2|
|      Hello|    2|
|        the|    2|
|       word|    2|
| streaming.|    2|
|     sample|    2|
|      world|    2|
|       This|    2|
|      Spark|    2|
|

In [ ]:
# sql adaptive query execution adaptive.coalescePartitions.enabled will makr paritions dynamic
sc.setLogLevel("Error")
#spark.conf.set("spark.sql.shuffle.partitions",3)
#spark.conf.get("spark.sql.shuffle.partitions")
#spark.conf.set("spark.sql.adaptive.enabled","false")
#spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled","false")
#spark.conf.set("spark.sql.execution.arrow.enabled",True)
#spark.conf.set("spark.sql.execution.arrow.fallback.enabled",True)

"""
Kafka integration
InPyspark - As integration between spark and kafka is lower, use 60 sec as interval
"""

#WithoutInferSchema
#headerTrue will read first row and assign column names but type is String for all
#spark job created to read first column
filepath = "file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/"

#message = kafkaUtils.createDirectStream(ssc,topics=['testtopic'],
#                                       kafkaParams={"message.broker.list":"localhost:9092"})

streaming_df = spark.readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "testtopic") \
  .load()

#ds = df \
#  .selectExpr("CAST(key AS STRING)") \
#  .writeStream \
#  .format("kafka") \
#  .option("kafka.bootstrap.servers", "localhost:9092") \
#  .option("topic", "testtopic") \
#  .start()

words = streaming_df.selectExpr("CAST(value AS STRING)")

word_df = words.select(explode(split(col("value"), " ")).alias("word"))

# Perform word count
word_count = word_df.groupBy("word").count()

# Write the word count DataFrame to the console
query = word_count.writeStream \
    .outputMode("complete") \
    .foreachBatch(lambda df, epoch_id: df.show()) \
    .start()

query.awaitTermination()

In [ ]:
query.stop()